In [1]:
import pandas as pd
import numpy as np

# # Чтение данных об арбитражных делах
# data_dela = pd.read_excel('data/СПАРК_Выборка_арбитражных_дел_20231014_2307.xlsx', skiprows=3)
# # Convert 'data_dela' DataFrame to Parquet format
# data_dela.drop('№', axis=1, inplace=True)
# 
# data_dela.to_parquet('data/data_dela.parquet', index=False)

# Read the Parquet file back into a DataFrame
data_dela = pd.read_parquet('data/data_dela.parquet')

# Заполнение пропущенных значений в столбце 'Дата иска' методом forward fill
data_dela['Дата иска'] = data_dela['Дата иска'].fillna(method='ffill')

# # Чтение данных о компаниях за разные года
# data_2008 = pd.read_excel('data/СПАРК_Выборка_компаний_20231021_2232.xlsx', skiprows=3)
# data_2013 = pd.read_excel('data/СПАРК_Выборка_компаний_20231021_2234.xlsx', skiprows=3)
# data_2019 = pd.read_excel('data/СПАРК_Выборка_компаний_20231021_2230.xlsx', skiprows=3)
# 
# data_2008 = data_2008.drop('№', axis=1)
# data_2013 = data_2013.drop('№', axis=1)
# data_2019 = data_2019.drop('№', axis=1)
# 
# # Сохранение данных о компаниях за разные года в формате Parquet
# data_2008.to_parquet('data/data_2008.parquet', index=False)
# data_2013.to_parquet('data/data_2013.parquet', index=False)
# data_2019.to_parquet('data/data_2019.parquet', index=False)


# Чтение данных из файлов Parquet
data_2008 = pd.read_parquet('data/data_2008.parquet')
data_2013 = pd.read_parquet('data/data_2013.parquet')
data_2019 = pd.read_parquet('data/data_2019.parquet')

In [2]:

# Обновление столбца 'Дата' в данных о компаниях на основе данных об арбитражных делах
for index, row in data_dela.iterrows():
    value_to_match = row['Ответчик']
    matching_row = data_2008[data_2008['Наименование'] == value_to_match]

    if not matching_row.empty:
        matching_index = matching_row.index[0]
        data_2008.at[matching_index, 'Дата иска'] = row['Дата иска']

In [3]:
col  = ['Наименование', 'Регистрационный номер', 'Адрес (место нахождения)',
        'Код налогоплательщика', 'Регион регистрации',
        'Вид деятельности/отрасль', 'ИФР', 'ИПД', 'Важная информация',
        'Мои списки',]
# Объединение данных по столбцам "Наименование" и "Регистрационный номер"
merged_data = pd.merge(data_2008, data_2013, on=col, how='inner')
merged_data = pd.merge(merged_data, data_2019, on=col, how='inner')
merged_data
# В результате получится объединенный DataFrame с дополнительными столбцами для каждого года


,Наименование,Регистрационный номер,Адрес (место нахождения),Код налогоплательщика,Регион регистрации,Вид деятельности/отрасль,ИФР,ИПД,Важная информация,Мои списки,...,"2018, Коэффициент быстрой ликвидности, %","2019, Коэффициент быстрой ликвидности, %","2020, Коэффициент быстрой ликвидности, %","2021, Коэффициент быстрой ликвидности, %","2022, Коэффициент быстрой ликвидности, %","2018, Коэффициент абсолютной ликвидности, %","2019, Коэффициент абсолютной ликвидности, %","2020, Коэффициент абсолютной ликвидности, %","2021, Коэффициент абсолютной ликвидности, %","2022, Коэффициент абсолютной ликвидности, %"
0,"100РА ЛТД, ООО",1.135038e+12,"119121, г. Москва, вн.тер.г. муниципальный окр...",5.038096e+09,Москва,Строительство жилых и нежилых зданий,59.0,NaN,\n\n\n,"""STR""",...,2.9346,0.6877,0.6952,0.6034,0.4631,0.3779,0.1368,0.0400,0.0128,0.0024
1,"1015 УСМР, АО",1.027739e+12,"143003, Московская обл, Одинцово г, г.о. Одинц...",7.734009e+09,Московская область,Строительство жилых и нежилых зданий,26.0,NaN,\nЕсть заявление в суд о банкротстве (13.07.20...,"""STR""",...,0.9701,0.9422,1.0669,1.0184,0.9065,0.0560,0.0307,0.0484,0.0199,0.0209
2,"162-КЖИ, ООО",1.143668e+12,"123308, г. Москва, вн.тер.г. муниципальный окр...",3.664135e+09,Москва,Строительство жилых и нежилых зданий,83.0,NaN,Регистрирующим органом принято решение о предс...,"""STR""",...,1.2777,1.1624,1.8047,14.3377,NaN,1.1154,0.2247,0.1754,0.0172,NaN
3,"1МСМУ СТАЛЬМОНТАЖ, ООО",1.077760e+12,"129329, г. Москва, вн.тер.г. муниципальный окр...",7.705809e+09,Москва,Строительство жилых и нежилых зданий,94.0,NaN,\n\n\n,"""STR""",...,1.0113,1.0202,0.9890,0.8961,0.8951,0.1125,0.0502,0.0761,0.0124,0.0123
4,"201 УНР, ООО",1.157746e+12,"117630, г. Москва, вн.тер.г. муниципальный окр...",7.719093e+09,Москва,Строительство жилых и нежилых зданий,37.0,NaN,\n\n\n,"""STR""",...,0.7878,0.6167,0.5593,0.5710,0.9376,0.3181,0.0265,0.0392,0.0599,0.3692
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9116,None,NaN,None,NaN,None,None,NaN,NaN,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9117,None,NaN,None,NaN,None,None,NaN,NaN,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9118,None,NaN,None,NaN,None,None,NaN,NaN,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9119,None,NaN,None,NaN,None,None,NaN,NaN,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:

# Удаление дубликатов по столбцу 'Наименование'
combined_data_unique = merged_data

# Добавление идентификаторов 'ID' к уникальным компаниям
combined_data_unique['ID'] = range(1, len(combined_data_unique) + 1)

# Выбор столбцов без цифр в названиях
columns_without_digits = combined_data_unique.filter(regex='^[^\d]*$')

# Выбор определенных столбцов
selected_columns = columns_without_digits.columns

# Транспонирование только выбранных столбцов
attribute_value_pairs = combined_data_unique.melt(id_vars=selected_columns, var_name='Атрибут', value_name='Значение')

In [5]:

# Разделение столбца 'Атрибут' на два столбца
split_values = attribute_value_pairs['Атрибут'].str.split(',', n=1, expand=True)
attribute_value_pairs['Год'] = split_values[0]
attribute_value_pairs['Атрибут2'] = split_values[1]

# Удаление исходного столбца 'Атрибут'
attribute_value_pairs.drop(['Атрибут', 'ID', 'ИПД', 'ИФР'], axis=1, inplace=True)

# Получение списка всех столбцов в DataFrame
all_columns = attribute_value_pairs.columns

# Исключение столбцов 'Атрибут2' и 'Значение' из списка
selected_columns = [col for col in all_columns if col not in ['Атрибут2', 'Значение']]

# Заполнение пропущенных значений в выбранных столбцах
attribute_value_pairs[selected_columns] = attribute_value_pairs[selected_columns].fillna('Заменить_значение_по_умолчанию')

In [6]:

# Применение pivot_table с использованием выбранных столбцов в качестве индекса
integrated_data = attribute_value_pairs.pivot_table(index=selected_columns, columns='Атрибут2', values='Значение').reset_index()

# Сброс имен индекса для улучшения читаемости
integrated_data.reset_index(inplace=True)

# Вывод результатов
integrated_data

Атрибут2,index,Наименование,Регистрационный номер,Адрес (место нахождения),Код налогоплательщика,Регион регистрации,Вид деятельности/отрасль,Важная информация,Мои списки,Дата иска,...,"Период погашения кредиторской задолженности, дни","Рентабельность активов (ROA), %","Рентабельность затрат, %","Рентабельность капитала (ROE), %","Рентабельность продаж, %","Себестоимость продаж, RUB","Собственный оборотный капитал, RUB","Соотношение совокупного долга к EBIT, %","Соотношение совокупного долга к капиталу, %","Соотношение чистого долга к капиталу, %"
0,0,"100РА ЛТД, ООО",1135038000679.0,"119121, г. Москва, вн.тер.г. муниципальный окр...",5.038096e+09,Москва,Строительство жилых и нежилых зданий,\n\n\n,"""STR""",Заменить_значение_по_умолчанию,...,48.21,0.7014,0.1629,2.4167,0.1401,2775000.0,342000.0,0.0796,0.0962,-0.1538
1,1,"100РА ЛТД, ООО",1135038000679.0,"119121, г. Москва, вн.тер.г. муниципальный окр...",5.038096e+09,Москва,Строительство жилых и нежилых зданий,\n\n\n,"""STR""",Заменить_значение_по_умолчанию,...,93.43,-0.2829,0.0225,-1.0439,0.0220,2096000.0,98000.0,NaN,0.0000,-0.2653
2,2,"100РА ЛТД, ООО",1135038000679.0,"119121, г. Москва, вн.тер.г. муниципальный окр...",5.038096e+09,Москва,Строительство жилых и нежилых зданий,\n\n\n,"""STR""",Заменить_значение_по_умолчанию,...,144.05,0.0308,0.0483,0.2192,0.0460,1699000.0,121000.0,0.0000,0.0000,-3.1488
3,3,"100РА ЛТД, ООО",1135038000679.0,"119121, г. Москва, вн.тер.г. муниципальный окр...",5.038096e+09,Москва,Строительство жилых и нежилых зданий,\n\n\n,"""STR""",Заменить_значение_по_умолчанию,...,29.73,0.5661,0.0843,1.7658,0.0777,26989000.0,1954000.0,0.0000,0.0000,-0.1464
4,4,"100РА ЛТД, ООО",1135038000679.0,"119121, г. Москва, вн.тер.г. муниципальный окр...",5.038096e+09,Москва,Строительство жилых и нежилых зданий,\n\n\n,"""STR""",Заменить_значение_по_умолчанию,...,173.59,0.0248,0.0130,0.0744,0.0129,8440000.0,2205000.0,0.5291,0.0475,-0.0542
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77276,77276,"ЯСУ, ООО",1137604013172.0,"150040, Ярославская обл, г.о. город Ярославль,...",7.604248e+09,Ярославская область,Строительство жилых и нежилых зданий,\n\n\n,"""STR""",Заменить_значение_по_умолчанию,...,270.34,0.0062,-0.0667,0.5051,-0.0715,538535000.0,-106245000.0,NaN,NaN,-5.6802
77277,77277,"ЯСУ, ООО",1137604013172.0,"150040, Ярославская обл, г.о. город Ярославль,...",7.604248e+09,Ярославская область,Строительство жилых и нежилых зданий,\n\n\n,"""STR""",Заменить_значение_по_умолчанию,...,180.63,0.0122,0.0174,1.3191,0.0171,475278000.0,-26689000.0,NaN,NaN,-56.7717
77278,77278,"ЯСУ, ООО",1137604013172.0,"150040, Ярославская обл, г.о. город Ярославль,...",7.604248e+09,Ярославская область,Строительство жилых и нежилых зданий,\n\n\n,"""STR""",Заменить_значение_по_умолчанию,...,75.47,0.0045,0.0037,0.8367,0.0037,572141000.0,24473000.0,1.8116,7.4906,2.7856
77279,77279,"ЯСУ, ООО",1137604013172.0,"150040, Ярославская обл, г.о. город Ярославль,...",7.604248e+09,Ярославская область,Строительство жилых и нежилых зданий,\n\n\n,"""STR""",Заменить_значение_по_умолчанию,...,68.78,0.0355,-0.0295,1.1948,-0.0304,275635000.0,25007000.0,0.1550,0.1913,-4.5800


In [7]:
# Save the original_data_filtered DataFrame to an Excel file
integrated_data.to_excel('data/integrated_data.xlsx', index=False)
# Save the original_data_filtered DataFrame to a Parquet file
integrated_data['Регистрационный номер'] = integrated_data['Регистрационный номер'].replace('Заменить_значение_по_умолчанию', 0)
integrated_data['Дата иска'] = integrated_data['Дата иска'].replace('Заменить_значение_по_умолчанию', 0)
# Convert 'Дата иска' column to datetime64 data type
integrated_data['Дата иска'] = pd.to_datetime(integrated_data['Дата иска'])
integrated_data.to_parquet('data/integrated_data.parquet', index=False)